<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'was',
 'diclofenac',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'limb',
 'cramp',
 'weakness',
 'sodium',
 'on',
 'by',
 'stomach',
 'depression',
 '-',
 'back',
 'is',
 'patients',
 'at',
 'cramps',
 'joint',
 'as',
 'upper',
 'gas',
 'that',
 'or',
 'memory',
 'leg',
 'drug',
 'mg',
 'legs',
 'pains',
 'headache',
 'knee',
 'from',
 'shoulder',
 'unable',
 'group',
 'not',
 'arthralgia',
 'study',
 'gastrointestinal',
 'treatment',
 'excessive',
 'neck',
 'this',
 'foot',
 'all',
 'arthritis',
 'aches',
 'after',
 'hip',
 'numbness',
 'insomnia',
 'voltaren',
 'disease',
 'p',
 'arthrotec',
 'release',
 'nausea',
 'bleeding',
 '/',
 'increased',
 'asthenia',
 'an',
 'like',
 'both',
 'skin',
 'be',
 'abdominal',
 'heart',
 'dizziness',
 'walk',
 'swelling',
 'tired',
 'no',
 

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.771179,3.382782,0.457117,29:39


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.416033,3.324533,0.461861,29:03


In [25]:
learn.save_encoder('pubmed_ask_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.978993,3.715057,0.344379,01:12


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.885437,3.078112,0.485207,01:18


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.293743,2.701613,0.547929,03:13


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.922352,2.354914,0.607101,03:31
1,1.676654,2.202471,0.635503,03:38


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.535163,2.012247,0.656805,03:24
1,1.337011,1.909709,0.680473,03:17


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.168222,1.806810,0.701775,03:16
1,1.026631,1.696368,0.713609,03:12


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.994444,1.619531,0.724260,03:12
1,0.877064,1.526480,0.737278,03:42


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.830623,1.511378,0.736095,03:16
1,0.765428,1.437904,0.745562,03:29


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.672069,1.412784,0.749112,03:27
1,0.673682,1.403528,0.750296,03:12
2,0.572673,1.332340,0.758580,03:50
3,0.533162,1.310211,0.765680,03:55


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.472662,1.337235,0.765680,03:21
1,0.497571,1.322984,0.775148,03:36
2,0.441795,1.250648,0.771598,03:34
3,0.407280,1.268818,0.785799,03:43


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.393233,1.277237,0.779882,03:44
1,0.392933,1.318423,0.782248,03:35
2,0.360094,1.288081,0.789349,03:39
3,0.320138,1.261729,0.786982,03:40


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.297891,1.295933,0.796450,03:19
1,0.349128,1.337193,0.795266,03:29
2,0.296819,1.316970,0.790533,03:31
3,0.245302,1.263289,0.791716,03:23


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.257867,1.362369,0.803550,03:13
1,0.319095,1.354300,0.790533,03:31
2,0.249937,1.308148,0.802367,03:31
3,0.218417,1.293644,0.801183,03:17


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.209840,1.304806,0.801183,03:34
1,0.279497,1.428750,0.795266,03:41
2,0.243027,1.315987,0.807101,03:29
3,0.209800,1.333722,0.798817,03:37


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.223140,1.350861,0.801183,03:15
1,0.240954,1.349093,0.798817,03:24
2,0.229958,1.325502,0.811834,03:36
3,0.193937,1.361184,0.808284,03:28


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.196885,1.460974,0.804734,03:27
1,0.183523,1.437552,0.798817,03:33


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.215325,1.414734,0.800000,03:41
1,0.197482,1.385448,0.802367,03:36


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.194634,1.361800,0.810651,03:39


In [44]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold0')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[49, 2, 343, 428, 5, 8, 8, 8, 8, 8, 8, 8, 509, 5, 625, 22, 405, 27, 27, 542, 29, 73, 211, 30, 31, 31, 32, 34, 34, 37, 37, 77, 37, 37, 37, 37, 218, 44, 537, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 53, 830, 58, 623, 71, 71, 74, 76, 76, 633, 79, 79, 79, 614, 379, 900, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 525, 111, 289, 79, 79, 80, 80, 80, 80, 80, 80, 80, 80, 80, 88, 89, 268, 137, 90, 91, 92, 94, 94, 94, 94, 94, 94, 94, 94, 96, 96, 98, 683, 98, 320, 887, 100, 100, 100, 100, 100, 100, 100, 100, 291, 337, 108, 660, 108, 636, 873, 111, 111, 111, 279, 111, 698, 117, 120, 1009, 1009, 122, 125, 900, 125, 125, 125, 125, 125, 125, 125, 125, 125, 777, 127, 129, 622, 745, 142, 142, 144, 794, 827, 151, 151, 899, 144, 157, 158, 165, 165, 166, 166, 166, 405, 172, 172, 172, 172, 172, 172, 172, 689, 389, 182, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 184, 184, 185, 187, 18

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

631
0.7277970011534025
